# Credit Card Fraud Detection

Este conjunto de dados contém as transações bancárias reais feitas pelos titulares de cartões europeus no ano de 2013. Por uma questão de segurança, as variáveis ​​reais não estão sendo compartilhadas, mas foram transformadas em versões do PCA. Como resultado, podemos encontrar 29 colunas de recursos e 1 coluna de classe final.

In [32]:
# Importando bibliotecas.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
# Importando dados.
data = pd.read_csv("creditcard.csv")

In [3]:
# Visualizando informações sobre os dados.
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [12]:
transacoes_totais = len(data)
normal = len(data[data.Class == 0])
fraudulenta = len(data[data.Class == 1])
porc_fraudulentas = round(fraudulenta/normal*100, 2)

print("O número total de transações é: {}".format(transacoes_totais))
print("Número de transações normais é: {}".format(normal))
print("Número de transações fraudulentas é: {}".format(fraudulenta))
print("Porcentagem de transações fraudulentas é: {}%".format(porc_fraudulentas))

O número total de transações é: 284807
Número de transações normais é: 284315
Número de transações fraudulentas é: 492
Porcentagem de transações fraudulentas é: 0.17%


In [6]:
min(data["Amount"]), max(data["Amount"])

(0.0, 25691.16)

In [15]:
# Escalando a variável.
sc = StandardScaler()
amount = data["Amount"].values
data["Amount"] = sc.fit_transform(amount.reshape(-1,1))

In [17]:
# Excluindo colunas 
data.drop(["Time"], axis = 1, inplace=True)

In [18]:
# Deletando colunas duplicadas.
data.drop_duplicates(inplace=True)

In [19]:
data.shape

(275663, 30)

In [20]:
# Treino e teste.
X = data.drop("Class", axis=1).values
y = data["Class"].values

In [23]:
# Separando os dados em treino e teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [25]:
# Construindo um modelo.
xgb = XGBClassifier(max_depth=4)
xgb.fit(X_train, y_train)
xgb_predict = xgb.predict(X_test)

[20:33:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [28]:
# Verificando a acurácia do modelo.
print('Pontuação de acurácia do modelo XGBoost é {}'.format(accuracy_score(y_test, xgb_predict)))

Pontuação de acurácia do modelo XGBoost é 0.9995465510674187


In [31]:
# Checando o F1-Score para o modelo.
print('F1-score do modelo XGBoost é {}'.format(f1_score(y_test, xgb_predict)))

F1-score do modelo XGBoost é 0.8407643312101911


In [34]:
# Matriz de confusão.
confusion_matrix(y_test, xgb_predict, labels = [0, 1])

array([[55042,     4],
       [   21,    66]], dtype=int64)

# Conclusão.

Nosso modelo tem 99.95% de acurácia, parte desta porcentagem deve-se a normalização dos dados. A coisa boa que notamos na matriz de confusão é que nosso modelo não está superajustado.